In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 14 17:18:19 2018

@author: xshitova
"""

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import xgboost as xgb 

from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

from matplotlib import pylab
from pylab import *

pylab.rcParams['figure.figsize'] = (10, 6)


limit_rows   = 7000000
dftest       = pd.read_csv("C:/Users/xshitova/Downloads/test_ver2.csv",delimiter=';', dtype={"sexo":str,
                                                    "ind_nuevo":str,
                                                    "ult_fec_cli_1t":str,
                                                    "indext":str}, nrows=limit_rows)
dftest.describe()

dftest["fecha_dato"] = pd.to_datetime(dftest["fecha_dato"],format="%Y-%m-%d")
dftest["fecha_alta"] = pd.to_datetime(dftest["fecha_alta"],format="%Y-%m-%d")
dftest["fecha_dato"].unique()

dftest["month"] = pd.DatetimeIndex(dftest["fecha_dato"]).month
dftest["age"]   = pd.to_numeric(dftest["age"], errors="coerce")



with sns.plotting_context("notebook",font_scale=1.5):
    sns.set_style("whitegrid")
    sns.distplot(dftest["age"].dropna(),
                 bins=80,
                 kde=False,
                 color="tomato")
    #sns.plt.title("Age Distribution")
    plt.ylabel("Count")

dftest.loc[dftest.age < 18,"age"]  = dftest.loc[(dftest.age >= 18) & (dftest.age <= 30),"age"].mean(skipna=True)
dftest.loc[dftest.age > 100,"age"] = dftest.loc[(dftest.age >= 30) & (dftest.age <= 100),"age"].mean(skipna=True)
dftest["age"].fillna(dftest["age"].mean(),inplace=True)
dftest["age"]                  = dftest["age"].astype(int)
with sns.plotting_context("notebook",font_scale=1.5):
    sns.set_style("whitegrid")
    sns.distplot(dftest["age"].dropna(),
                 bins=80,
                 kde=False,
                 color="tomato")
    #sns.plt.title("Age Distribution")
    plt.ylabel("Count")
    plt.xlim((15,100))

b=dftest["ind_nuevo"].isnull().sum()

months_active = dftest.loc[dftest["ind_nuevo"].isnull(),:].groupby("ncodpers", sort=False).size()
months_active.max()
dftest.loc[dftest["ind_nuevo"].isnull(),"ind_nuevo"] = 1 

dftest.antiguedad = pd.to_numeric(dftest.antiguedad,errors="coerce")
np.sum(dftest["antiguedad"].isnull())
dftest.loc[dftest["antiguedad"].isnull(),"ind_nuevo"].describe()
dftest.loc[dftest.antiguedad.isnull(),"antiguedad"] = dftest.antiguedad.min()
dftest.loc[dftest.antiguedad <0, "antiguedad"]      = 0 # Thanks @StephenSmith for bug-find

dates=dftest.loc[:,"fecha_alta"].sort_values().reset_index()
median_date = int(np.median(dates.index.values))
dftest.loc[dftest.fecha_alta.isnull(),"fecha_alta"] = dates.loc[median_date,"fecha_alta"]
dftest["fecha_alta"].describe()

pd.Series([i for i in dftest.indrel]).value_counts()
dftest.loc[dftest.indrel.isnull(),"indrel"] = 1 

dftest.drop(["tipodom","cod_prov"],axis=1,inplace=True)

np.sum(dftest["ind_actividad_cliente"].isnull())
dftest.loc[dftest.ind_actividad_cliente.isnull(),"ind_actividad_cliente"] = \
dftest["ind_actividad_cliente"].median()

dftest.loc[dftest.nomprov=="CORU\xc3\x91A, A","nomprov"] = "CORUNA, A" 
dftest.loc[dftest.nomprov.isnull(),"nomprov"] = "UNKNOWN"

incomes = dftest.loc[dftest.renta.notnull(),:].groupby("nomprov").agg({"renta":{"MedianIncome":median}})
incomes.sort_values(by=("renta","MedianIncome"),inplace=True)
incomes.reset_index(inplace=True)
incomes.nomprov = incomes.nomprov.astype("category", categories=[i for i in dftest.nomprov.unique()],ordered=False)
incomes.head()

grouped        = dftest.groupby("nomprov").agg({"renta":lambda x: x.median(skipna=True)}).reset_index()
new_incomes    = pd.merge(dftest,grouped,how="inner",on="nomprov").loc[:, ["nomprov","renta_y"]]
new_incomes    = new_incomes.rename(columns={"renta_y":"renta"}).sort_values("renta").sort_values("nomprov")
dftest.sort_values("nomprov",inplace=True)
dftest             = dftest.reset_index()
new_incomes    = new_incomes.reset_index()

#fill in the rentas - median by provinces 
dftest.loc[dftest.renta.isnull(),"renta"] = new_incomes.loc[dftest.renta.isnull(),"renta"].reset_index()
dftest.loc[dftest.renta.isnull(),"renta"] = dftest.loc[dftest.renta.notnull(),"renta"].median()
dftest.sort_values(by="fecha_dato",inplace=True)

string_data = dftest.select_dtypes(include=["object"])
missing_columns = [col for col in string_data if string_data[col].isnull().any()]
for col in missing_columns:
    print("Unique values for {0}:\n{1}\n".format(col,string_data[col].unique()))
del string_data

dftest.loc[dftest.indfall.isnull(),"indfall"] = "N"
dftest.loc[dftest.tiprel_1mes.isnull(),"tiprel_1mes"] = "A"
dftest.tiprel_1mes = dftest.tiprel_1mes.astype("category")

# As suggested by @StephenSmith
map_dict = { 1.0  : "1",
            "1.0" : "1",
            "1"   : "1",
            "3.0" : "3",
            "P"   : "P",
            3.0   : "3",
            2.0   : "2",
            "3"   : "3",
            "2.0" : "2",
            "4.0" : "4",
            "4"   : "4",
            "2"   : "2"}

dftest.indrel_1mes.fillna("P",inplace=True)
dftest.indrel_1mes = dftest.indrel_1mes.apply(lambda x: map_dict.get(x,x))
dftest.indrel_1mes = dftest.indrel_1mes.astype("category")


unknown_cols = [col for col in missing_columns if col not in ["indfall","tiprel_1mes","indrel_1mes"]]
for col in unknown_cols:
    dftest.loc[dftest[col].isnull(),col] = "UNKNOWN"

print(dftest.isnull().any())

dftest.to_csv("C:/Users/xshitova/Documents/Data science/santander_results_before_modelisation_test.csv")
#the end of cleaning and imputation  for test

 


